In [ ]:
pcpath = "../input/california-house-price/housing_train.csv"

import numpy as np
import pandas as pd
# loading from url cause errors
data = pd.read_csv(pcpath, sep=',')

In [ ]:
test_pcpath = "../input/california-house-price/housing_train.csv"
test_data = pd.read_csv(test_pcpath, sep=',')
test_data.head(2)

In [ ]:
test_anspath = "../input/california-house-price/housing_test_sample_solution.csv"
test_ans = pd.read_csv(test_anspath)
test_ans.head(2)

In [ ]:
data["ocean_proximity"].unique()

In [ ]:
print('~%.2f%% of data is corrupted' % ((data.notna().sum(axis=1) < 10).sum(axis=0) / data.shape[0] * 100))

In [ ]:
for i in data.keys()[1:]:
    print(i, (data[i].notna() == False).sum(axis=0))

In [ ]:
train_data = data[data.notna().sum(axis=1) == 10]
train_data["ocean_proximity"] = pd.factorize(train_data["ocean_proximity"])[0]
train_data.head(5)

In [ ]:
a = np.asarray(train_data.corr())
corr_matrix = a - (np.indices((10, 10))[0] == np.indices((10, 10))[1]).astype(np.float32)
print(np.round(np.max(corr_matrix, axis=0), decimals=2))
print(np.round(np.min(corr_matrix, axis=0), decimals=2))
train_data.corr().round(decimals=2)

In [ ]:
from sklearn.linear_model import LinearRegression
all_other_keys = list(filter(lambda x: x not in  ['median_house_value'], train_data.keys()))
a = []
for i in all_other_keys:
    lnreg = LinearRegression(fit_intercept = True, normalize=True, n_jobs=-1)
    train_len = int(train_data.shape[0] * 9 / 10)
    lnreg.fit(np.asarray(train_data[i][:train_len]).reshape(-1, 1), np.asarray(train_data['median_house_value'][:train_len]))
    a += [np.sqrt(np.mean(np.square(np.asarray(train_data['median_house_value'][train_len:]) -lnreg.predict(np.asarray(train_data[i][train_len:]).reshape(-1, 1)))))]
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 7))
plt.plot(a, 'o')
plt.xticks(np.arange(len(all_other_keys)), all_other_keys)
plt.show()
print(a)

In [ ]:
def ols(X, Y):
    return np.linalg.inv(X.T @ X) @ X.T @ Y
def errorl2(X, Y, w):
    X, Y, w = np.array(X), np.array(Y), np.array(w)
    return np.mean(np.square(X @ w - Y))
def errorl1(X, Y, w):
    X, Y, w = np.array(X), np.array(Y), np.array(w)
    return np.mean(np.abs(X @ w - Y))

In [ ]:
other_keys = train_data.keys()[train_data.keys() != 'total_bedrooms']
N = train_data.shape[0]
_X = train_data[other_keys]
Y = train_data['total_bedrooms']
X = _X

print('all keys %.2f %.2f' % (errorl2(X, Y, ols(X, Y)), errorl1(X, Y, ols(X, Y))))
__X = np.array(_X)
for i in range(9):
    X = __X[:,i:i+1]
    print(other_keys[i], '%.2f %.2f' % (errorl2(X, Y, ols(X, Y)), errorl1(X, Y, ols(X, Y))))
X = _X[['total_rooms', 'population', 'households']]
X = np.array(X)
print('total_rooms, population, households %.2f %.2f' % (errorl2(X, Y, ols(X, Y)), errorl1(X, Y, ols(X, Y))))

In [ ]:
test_d = test_data[test_data.keys()[1:]]
test_d["ocean_proximity"] = pd.factorize(test_d["ocean_proximity"])[0]
other_keys = test_d.keys()[test_d.keys() != 'total_bedrooms']
bad_rows = test_d.notna().sum(axis=1) < 9
good_rows = bad_rows == False
w = ols(test_d.loc[good_rows][other_keys], test_d.loc[good_rows]['total_bedrooms'])
test_d.loc[bad_rows, 'total_bedrooms'] = np.array(test_d.loc[bad_rows][other_keys]) @ w
x_test = np.array(test_d)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
other_keys = train_data.keys()[train_data.keys() != "median_house_value"]
x_train, y_train = np.array(train_data[other_keys]), np.array(train_data['median_house_value'])
x_test = np.array(test_d)

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import xgboost as xgb

from pandas.plotting import scatter_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from geopy.distance import geodesic

import gc

housing = pd.read_csv(r"../input/california-house-price/housing_train.csv")
city_lat_long = pd.read_csv(r"../input/california-housing-feature-engineering/cal_cities_lat_long.csv")
city_pop_data = pd.read_csv(r"../input/california-housing-feature-engineering/cal_populations_city.csv")
county_pop_data = pd.read_csv(r"../input/california-housing-feature-engineering/cal_populations_county.csv")

city_coords = {}

for dat in city_lat_long.iterrows():
    row = dat[1]
    if row['Name'] not in city_pop_data['City'].values:   
        continue           
    else: 
        city_coords[row['Name']] = (float(row['Latitude']), float(row['Longitude']))
        
newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
x = geodesic(newport_ri, cleveland_oh)
x #distance stored in km, see units on printing
print(x)
type(x.kilometers) #is it a float?

def closest_point(location, location_dict):
    """ take a tuple of latitude and longitude and 
        compare to a dictonary of locations where
        key = location name and value = (lat, long)
        returns tuple of (closest_location , distance) """
    closest_location = None
    for city in location_dict.keys():
        distance = geodesic(location, location_dict[city]).kilometers
        if closest_location is None:
            closest_location = (city, distance)
        elif distance < closest_location[1]:
            closest_location = (city, distance)
    return closest_location
  
test = (39.524325, -122.293592) #likely 'Willows'
closest_point(test, city_coords)

city_pop_dict = {}
for dat in city_pop_data.iterrows():
	row = dat[1]
	city_pop_dict[row['City']] =  row['pop_april_1990']


big_cities = {}

for key, value in city_coords.items():
    if city_pop_dict[key] > 500000:
        big_cities[key] = value
    
from pandas import Series

def progress_coroutine(print_on = 10000):
    print("Starting progress monitor")

    iterations = 0
    while True:
        yield
        iterations += 1
        if (iterations % print_on == 0):
            print("{} iterations done".format(iterations))

def percentage_coroutine(to_process, print_on_percent = 0.001):
    print("Starting progress percentage monitor")

    processed = 0
    count = 0
    print_count = to_process*print_on_percent
    while True:
        yield
        processed += 1
        count += 1
        if (count >= print_count):
            count = 0
            pct = (float(processed)/float(to_process) * 100)

            print("{}% finished".format(pct))

def trace_progress(func, progress = None):
    def callf(*args, **kwargs):
        if (progress is not None):
            progress.send(None)

        return func(*args, **kwargs)

    return callf


def f(x):
    return closest_point((x['latitude'],x['longitude']),city_coords)
co2 = percentage_coroutine(len(housing))
co2.__next__()

housing['close_city'] = housing.apply(trace_progress(f, progress = co2), axis = 1)
housing['close_city_name'] = [x[0] for x in housing['close_city'].values]
housing['close_city_dist'] = [x[1] for x in housing['close_city'].values]
housing['close_city_pop'] = [city_pop_dict[x] for x in housing['close_city_name'].values]

housing = housing.drop('close_city', axis=1)
housing.head()


#add the data relating to the points to the closest big city
def g(x):
    return closest_point((x['latitude'],x['longitude']),big_cities)
co2 = percentage_coroutine(len(housing))
co2.__next__()
housing['big_city'] = housing.apply(trace_progress(g, progress = co2), axis = 1)
housing['big_city_name'] = [x[0] for x in housing['big_city'].values]
housing['big_city_dist'] = [x[1] for x in housing['big_city'].values]

housing = housing.drop('big_city', axis=1)

housing.to_csv(r"./new_geo_data_train.csv")
_new_geo_data_train = housing.copy(deep=True)

In [ ]:
housing = pd.read_csv(r"../input/california-house-price/housing_test.csv")
city_lat_long = pd.read_csv(r"../input/california-housing-feature-engineering/cal_cities_lat_long.csv")
city_pop_data = pd.read_csv(r"../input/california-housing-feature-engineering/cal_populations_city.csv")
county_pop_data = pd.read_csv(r"../input/california-housing-feature-engineering/cal_populations_county.csv")

city_coords = {}

for dat in city_lat_long.iterrows():
    row = dat[1]
    if row['Name'] not in city_pop_data['City'].values:   
        continue           
    else: 
        city_coords[row['Name']] = (float(row['Latitude']), float(row['Longitude']))
        
newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
x = geodesic(newport_ri, cleveland_oh)
x #distance stored in km, see units on printing
print(x)
type(x.kilometers) #is it a float?

def closest_point(location, location_dict):
    """ take a tuple of latitude and longitude and 
        compare to a dictonary of locations where
        key = location name and value = (lat, long)
        returns tuple of (closest_location , distance) """
    closest_location = None
    for city in location_dict.keys():
        distance = geodesic(location, location_dict[city]).kilometers
        if closest_location is None:
            closest_location = (city, distance)
        elif distance < closest_location[1]:
            closest_location = (city, distance)
    return closest_location
  
test = (39.524325, -122.293592) #likely 'Willows'
closest_point(test, city_coords)

city_pop_dict = {}
for dat in city_pop_data.iterrows():
	row = dat[1]
	city_pop_dict[row['City']] =  row['pop_april_1990']


big_cities = {}

for key, value in city_coords.items():
    if city_pop_dict[key] > 500000:
        big_cities[key] = value
    
from pandas import Series

def progress_coroutine(print_on = 10000):
    print("Starting progress monitor")

    iterations = 0
    while True:
        yield
        iterations += 1
        if (iterations % print_on == 0):
            print("{} iterations done".format(iterations))

def percentage_coroutine(to_process, print_on_percent = 0.001):
    print("Starting progress percentage monitor")

    processed = 0
    count = 0
    print_count = to_process*print_on_percent
    while True:
        yield
        processed += 1
        count += 1
        if (count >= print_count):
            count = 0
            pct = (float(processed)/float(to_process) * 100)

            print("{}% finished".format(pct))

def trace_progress(func, progress = None):
    def callf(*args, **kwargs):
        if (progress is not None):
            progress.send(None)

        return func(*args, **kwargs)

    return callf


def f(x):
  return closest_point((x['latitude'],x['longitude']),city_coords)
co2 = percentage_coroutine(len(housing))
co2.__next__()

housing['close_city'] = housing.apply(trace_progress(f, progress = co2), axis = 1)
housing['close_city_name'] = [x[0] for x in housing['close_city'].values]
housing['close_city_dist'] = [x[1] for x in housing['close_city'].values]
housing['close_city_pop'] = [city_pop_dict[x] for x in housing['close_city_name'].values]

housing = housing.drop('close_city', axis=1)
housing.head()


#add the data relating to the points to the closest big city
def g(x):
  return closest_point((x['latitude'],x['longitude']),big_cities)
co2 = percentage_coroutine(len(housing))
co2.__next__()
housing['big_city'] = housing.apply(trace_progress(g, progress = co2), axis = 1)
housing['big_city_name'] = [x[0] for x in housing['big_city'].values]
housing['big_city_dist'] = [x[1] for x in housing['big_city'].values]

housing = housing.drop('big_city', axis=1)

housing.to_csv(r"./new_geo_data_test.csv")
_new_geo_data_test = housing.copy(deep=True)

In [ ]:
_new_geo_data_test = housing.copy(deep=True)

In [ ]:
with open('./b.txt', 'w') as f:
    f.write('1')


In [ ]:
from sklearn.model_selection import KFold

pcpath = r"./new_geo_data.csv"
# ndata = pd.read_csv(pcpath, sep=',')
ndata = _new_geo_data_train
big_city_name = ndata['big_city_name'].unique()
ocean_proximity = ndata["ocean_proximity"].unique()
print(ocean_proximity)
print(big_city_name)
other_keys = list(filter(lambda x: x not in  ['Unnamed: 0', 'close_city_name', 'big_city_name', 'income_cat'], ndata.keys()))
ndata_train = ndata[other_keys]
ndata_train.loc[:, "ocean_proximity"] = pd.factorize(ndata_train["ocean_proximity"])[0]
ndata_train = ndata_train[ndata_train.keys()[ndata_train.keys() != 'ocean_proximity']]
for i in big_city_name[:-1]:
  ndata_train[i] = ndata['big_city_name'] == i
for i in ocean_proximity[:-1]:
  ndata_train[i] = ndata['ocean_proximity'] == i

other_keys = list(filter(lambda x: x not in  ['total_bedrooms', 'median_house_value'], ndata_train.keys()))
bad_rows = ndata_train.notna().sum(axis=1) < ndata_train.shape[1]
good_rows = bad_rows == False
ln_room = LinearRegression(fit_intercept = True, normalize=True, n_jobs=-1)
ln_room.fit(ndata_train.loc[good_rows, other_keys], ndata_train.loc[good_rows, 'total_bedrooms'])
ndata_train.loc[bad_rows, 'total_bedrooms'] = ln_room.predict(ndata_train.loc[bad_rows, other_keys])

other_keys = ndata_train.keys()[ndata_train.keys() != 'median_house_value']
ndata_train = ndata_train.sort_index()
print(ndata_train[other_keys].head(1))
x_train = np.array(ndata_train[other_keys])
y_train = np.array(ndata_train['median_house_value']).reshape(-1, 1)
ln = LinearRegression(fit_intercept = True, normalize=True, n_jobs=-1)
x_test_len = int(x_train.shape[0] * 1 / 10)
kf = KFold(n_splits=10)
kf.get_n_splits(x_train)
np.random.seed(42)
idxs = np.arange(x_train.shape[0])
np.random.shuffle(idxs)
x_train, y_train = x_train[idxs], y_train[idxs]
i = 0
for train_index, test_index in kf.split(x_train):
    if i == 6:
        break
    print("TRAIN:", train_index, "TEST:", test_index)
    X_tr, X_te = x_train[train_index], x_train[test_index]
    y_tr, y_te = y_train[train_index], y_train[test_index]
    ln.fit(X_tr, y_tr)
    print("RMSE ==> ",np.sqrt(mean_squared_error(y_train,ln.predict(x_train))))
    print("RMSE ==> ",np.sqrt(mean_squared_error(y_tr,ln.predict(X_tr))))
    print("RMSE ==> ",np.sqrt(mean_squared_error(y_te,ln.predict(X_te))))
    i+=1
#print("RMSE ==> ",np.sqrt(mean_squared_error(y_train[:x_train_len],ln.predict(x_train[:x_train_len]))))
#print("RMSE ==> ",np.sqrt(mean_squared_error(y_train[x_train_len:],ln.predict(x_train[x_train_len:]))))

pcpath = r"./new_geo_data_test.csv"
ndata = _new_geo_data_test
# ndata = pd.read_csv(pcpath, sep=',')
other_keys = list(filter(lambda x: x not in  ['Id', 'Unnamed: 0', 'close_city_name', 'big_city_name'], ndata.keys()))
ndata_train = ndata[other_keys]
ndata_train.loc[:, "ocean_proximity"] = pd.factorize(ndata["ocean_proximity"])[0]
ndata_train = ndata_train[ndata_train.keys()[ndata_train.keys() != 'ocean_proximity']]
print(ndata['big_city_name'].unique())
print(ndata["ocean_proximity"].unique())
ocean_dict = {'INLAND':'INLAND', 'LessThan1h':'<1H OCEAN', '<1H OCEAN':'<1H OCEAN',
              'NEAR BAY':'NEAR BAY', 'NEAR OCEAN':'NEAR OCEAN', 'ISLAND':'ISLAND'}
for i in big_city_name[:-1]:
    ndata_train[i] = ndata['big_city_name'] == i
for i in ocean_proximity[:-1]:
    print(ocean_dict[i])
    ndata_train[i] = ndata['ocean_proximity'] == ocean_dict[i]
  
other_keys = ndata_train.keys()[ndata_train.keys() != 'total_bedrooms']
bad_rows = ndata_train.notna().sum(axis=1) < ndata_train.shape[1]
good_rows = bad_rows == False
print(ndata_train.head(1))
ndata_train.loc[bad_rows, 'total_bedrooms'] = ln_room.predict(ndata_train.loc[bad_rows, other_keys])

ndata_train = ndata_train.sort_index()
x_test = np.array(ndata_train)
pcpath = r"../input/california-house-price/housing_test_sample_solution.csv"
model_sol = pd.read_csv(pcpath, sep=',')
y_test_ans = ln.predict(x_test)
model_sol['median_house_value'] = y_test_ans.flatten()
model_sol.to_csv(r"/kaggle/working/model_solution.csv", index=False)

Финальный результат вычислялся на такой модели lightgbm.LGBMRegressor() снизу: 

In [ ]:
import lightgbm
lgb = lightgbm.LGBMRegressor()

from sklearn.model_selection import KFold

pcpath = r"C:\Users\danil\Desktop\6семестр_309\матпрак\data\new_geo_data.csv"
ndata = pd.read_csv(pcpath, sep=',')
big_city_name = ndata['big_city_name'].unique()
ocean_proximity = ndata["ocean_proximity"].unique()
print(ocean_proximity)
print(big_city_name)
other_keys = list(filter(lambda x: x not in  ['Unnamed: 0', 'close_city_name', 'big_city_name', 'income_cat'], ndata.keys()))
ndata_train = ndata[other_keys]
ndata_train.loc[:, "ocean_proximity"] = pd.factorize(ndata_train["ocean_proximity"])[0]
ndata_train = ndata_train[ndata_train.keys()[ndata_train.keys() != 'ocean_proximity']]
for i in big_city_name[:-1]:
  ndata_train[i] = ndata['big_city_name'] == i
for i in ocean_proximity[:-1]:
  ndata_train[i] = ndata['ocean_proximity'] == i

other_keys = list(filter(lambda x: x not in  ['total_bedrooms', 'median_house_value'], ndata_train.keys()))
bad_rows = ndata_train.notna().sum(axis=1) < ndata_train.shape[1]
good_rows = bad_rows == False
ln_room = LinearRegression(fit_intercept = True, normalize=True, n_jobs=-1)
ln_room.fit(ndata_train.loc[good_rows, other_keys], ndata_train.loc[good_rows, 'total_bedrooms'])
ndata_train.loc[bad_rows, 'total_bedrooms'] = ln_room.predict(ndata_train.loc[bad_rows, other_keys])

other_keys = ndata_train.keys()[ndata_train.keys() != 'median_house_value']
ndata_train = ndata_train.sort_index()
print(ndata_train[other_keys].head(1))
x_train = np.array(ndata_train[other_keys])
y_train = np.array(ndata_train['median_house_value']).reshape(-1, 1)
ln = LinearRegression(fit_intercept = True, normalize=True, n_jobs=-1)
x_test_len = int(x_train.shape[0] * 1 / 10)
kf = KFold(n_splits=10)
kf.get_n_splits(x_train)
np.random.seed(42)
idxs = np.arange(x_train.shape[0])
np.random.shuffle(idxs)
x_train, y_train = x_train[idxs], y_train[idxs]
i = 0
for train_index, test_index in kf.split(x_train):
    if i == 6:
        break
    print("TRAIN:", train_index, "TEST:", test_index)
    X_tr, X_te = x_train[train_index], x_train[test_index]
    y_tr, y_te = y_train[train_index], y_train[test_index]
    lgb.fit(X_tr, y_tr)
    print("RMSE ==> ",np.sqrt(mean_squared_error(y_train,lgb.predict(x_train))))
    print("RMSE ==> ",np.sqrt(mean_squared_error(y_tr,lgb.predict(X_tr))))
    print("RMSE ==> ",np.sqrt(mean_squared_error(y_te,lgb.predict(X_te))))
    i+=1
#print("RMSE ==> ",np.sqrt(mean_squared_error(y_train[:x_train_len],ln.predict(x_train[:x_train_len]))))
#print("RMSE ==> ",np.sqrt(mean_squared_error(y_train[x_train_len:],ln.predict(x_train[x_train_len:]))))

pcpath = r"C:\Users\danil\Desktop\6семестр_309\матпрак\data\new_geo_data_test.csv"
ndata = pd.read_csv(pcpath, sep=',')
other_keys = list(filter(lambda x: x not in  ['Id', 'Unnamed: 0', 'close_city_name', 'big_city_name'], ndata.keys()))
ndata_train = ndata[other_keys]
ndata_train.loc[:, "ocean_proximity"] = pd.factorize(ndata["ocean_proximity"])[0]
ndata_train = ndata_train[ndata_train.keys()[ndata_train.keys() != 'ocean_proximity']]
print(ndata['big_city_name'].unique())
print(ndata["ocean_proximity"].unique())
ocean_dict = {'INLAND':'INLAND', 'LessThan1h':'<1H OCEAN', 
              'NEAR BAY':'NEAR BAY', 'NEAR OCEAN':'NEAR OCEAN', 'ISLAND':'ISLAND'}
for i in big_city_name[:-1]:
  ndata_train[i] = ndata['big_city_name'] == i
for i in ocean_proximity[:-1]:
  print(ocean_dict[i])
  ndata_train[i] = ndata['ocean_proximity'] == ocean_dict[i]
  
other_keys = ndata_train.keys()[ndata_train.keys() != 'total_bedrooms']
bad_rows = ndata_train.notna().sum(axis=1) < ndata_train.shape[1]
good_rows = bad_rows == False
print(ndata_train.head(1))
ndata_train.loc[bad_rows, 'total_bedrooms'] = ln_room.predict(ndata_train.loc[bad_rows, other_keys])

ndata_train = ndata_train.sort_index()
x_test = np.array(ndata_train)
pcpath = r"C:\Users\danil\Desktop\6семестр_309\матпрак\data\housing_test_sample_solution.csv"
model_sol = pd.read_csv(pcpath, sep=',')
y_test_ans = lgb.predict(x_test)
model_sol['median_house_value'] = y_test_ans.flatten()
model_sol.to_csv(r"C:\Users\danil\Desktop\6семестр_309\матпрак\data\model_solution.csv", index=False)